# AML CLI v2 と YAML によるモデルのデプロイ

[00_LoadData.ipynb](./00_LoadData.ipynb) を実行して、あらかじめデータセットの登録をおこなってください。

> Note : データセットのアップロード・登録を CLI v2 と YAML で記述できます。

## Azure へのログイン

In [ ]:
!az login

In [ ]:
!az account set -s {AZURE_SUBSCRIPTION_ID}

## AML ワークスペースの環境設定

In [1]:
my_resource_group = "{AML-RESOURCE-GROUP-NAME}"
my_workspace = "{AML-WORSPACE-NAME}"

## Model 登録

下記コマンドで、[02_TrainOnRemote_CLI2.ipynb](./02_TrainOnRemote_CLI2.ipynb) でダウンロードした Model を AML に登録します。

In [2]:
!az ml model create --name arima_model \
  --version 1 \
  --path ./artifacts/outputs/arima_model.pkl \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Command group 'ml model' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Uploading arima_model.pkl (< 1 MB): 100%|██| 13.2k/13.2k [00:00<00:00, 1.06MB/s]


{
  "creation_context": {
    "created_at": "2022-04-26T05:29:42.744786+00:00",
    "created_by": "Tsuyoshi Matsuzaki",
    "created_by_type": "User",
    "last_modified_at": "2022-04-26T05:29:42.744786+00:00",
    "last_modified_by": "Tsuyoshi Matsuzaki",
    "last_modified_by_type": "User"
  },
  "id": "azureml:/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/TEST2020426-ML/providers/Microsoft.MachineLearningServices/workspaces/ws01/models/arima_model/versions/1",
  "name": "arima_model",
  "path": "azureml://subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/TEST2020426-ML/workspaces/ws01/datastores/workspaceblobstore/paths/LocalUpload/560e2b2e876016b60c2dd1068aa0b7dd/arima_model.pkl",
  "properties": {},
  "resourceGroup": "TEST2020426-ML",
  "tags"

## Managed Endpoint の作成

In [5]:
%%writefile managed_endpoint_config.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineEndpoint.schema.json
name: my-endpoint-test
auth_mode: key

Writing managed_endpoint_config.yml


In [6]:
!az ml online-endpoint create --file managed_endpoint_config.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Command group 'ml online-endpoint' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "auth_mode": "key",
  "id": "/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/TEST2020426-ML/providers/Microsoft.MachineLearningServices/workspaces/ws01/onlineEndpoints/my-endpoint-test",
  "identity": {
    "principal_id": "9c84fc38-cae5-4cf5-8559-adc95ab0ae5e",
    "tenant_id": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "type": "system_assigned"
  },
  "location": "eastus",
  "name": "my-endpoint-test",
  "properties": {
    "AzureAsyncOperationUri": "https://management.azure.com/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:584fe42d-825e-41e8-accd-dcdd7156ae4b:620cfff6-e610-425b-a917-f606b6c524bf?api-version=2021-10-01",
    "azureml.onlineendpointid": "/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourcegroups/test2020426-ml/providers/

## Deployment 作成とトラフィックの割り当て

In [7]:
%%writefile deployment_config.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
name: deployment-v1
endpoint_name: my-endpoint-test
model: azureml:arima_model@latest
code_configuration:
  code: scripts
  scoring_script: score.py
environment: 
  conda_file: scripts/arima-env.yml
  image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04
instance_type: Standard_DS2_v2
instance_count: 1

Writing deployment_config.yml


In [8]:
!az ml online-deployment create --file deployment_config.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace \
  --all-traffic

Command group 'ml online-deployment' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Check: endpoint my-endpoint-test exists
Creating/updating online deployment deployment-v1 ...............................................................Done (5m 44s)
{
  "app_insights_enabled": false,
  "code_configuration": {
    "code": "/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/TEST2020426-ML/providers/Microsoft.MachineLearningServices/workspaces/ws01/codes/831618de-8420-44d7-b2de-925151530c8c/versions/1",
    "scoring_script": "score.py"
  },
  "endpoint_name": "my-endpoint-test",
  "environment": "azureml:/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/TEST2020426-ML/providers/Microsoft.MachineLearningServices/workspaces/ws01/environments/CliV2AnonymousEnvironment/versions/f046b9ff-b8c0-4aeb-a08a-ffe2cac4eb16",
  "environment_variables": {},
  "instance_count": 1,
  "instance_type": "Standard_DS2_v2",
  "m